In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm co
                          nstant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni 
                          pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f
                          16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm

**Load The Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math

In [2]:
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
#LQ = 500       # Large Query: 500 queries
LQ = 1_000     # Large Query: 1K queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, LQ)
]

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**-----------------Ball Tree--------------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=220):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
#*-----------------------------------------------------------------------------

**Ball Tree Large Scale Examination**

In [ ]:
import time
from tqdm import tqdm

# Test code for Ball Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 50

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the Ball Tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of Ball Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50): 100%|██████████| 1000/1000 [00:07<00:00, 134.77it/s]


-------------------------------------------------------------
Build Time: 6.6163 sec, Search Time: 7.3560 sec
Leaf Size: 50, Total Time: 13.9723 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 100): 100%|██████████| 1000/1000 [00:07<00:00, 139.04it/s]


-------------------------------------------------------------
Build Time: 4.8920 sec, Search Time: 7.1292 sec
Leaf Size: 100, Total Time: 12.0212 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


...

-------------------------------------------------------------
Build Time: 2.1444 sec, Search Time: 18.1527 sec
Leaf Size: 9,900, Total Time: 20.2971 sec

199: === Testing with Leaf Size: 9,950, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 9,950): 100%|██████████| 1000/1000 [00:18<00:00, 54.40it/s]


-------------------------------------------------------------
Build Time: 2.0787 sec, Search Time: 18.2403 sec
Leaf Size: 9,950, Total Time: 20.3190 sec

200: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 10,000): 100%|██████████| 1000/1000 [00:18<00:00, 54.55it/s]

-------------------------------------------------------------
Build Time: 2.0802 sec, Search Time: 18.1898 sec
Leaf Size: 10,000, Total Time: 20.2700 sec

=== Summary of Ball Tree Node Size Test ===
Leaf Size: 50, Build Time: 6.6163 sec, Search Time: 7.3560 sec, Total Time: 13.9723 sec
Leaf Size: 100, Build Time: 4.8920 sec, Search Time: 7.1292 sec, Total Time: 12.0212 sec
Leaf Size: 150, Build Time: 4.3360 sec, Search Time: 6.9705 sec, Total Time: 11.3066 sec
Leaf Size: 200, Build Time: 4.0303 sec, Search Time: 6.8993 sec, Total Time: 10.9296 sec
Leaf Size: 250, Build Time: 3.8944 sec, Search Time: 6.8760 sec, Total Time: 10.7704 sec
Leaf Size: 300, Build Time: 3.6864 sec, Search Time: 6.7551 sec, Total Time: 10.4415 sec
Leaf Size: 350, Build Time: 3.5620 sec, Search Time: 6.7294 sec, Total Time: 10.2914 sec
Leaf Size: 400, Build Time: 3.4863 sec, Search Time: 6.6131 sec, Total Time: 10.0993 sec
Leaf Size: 450, Build Time: 3.4062 sec, Search Time: 6.6230 sec, Total Time: 10.0292 sec
L

**Ball Tree Large Scale (Run 2)**

In [ ]:
import time
from tqdm import tqdm

# Test code for Ball Tree
results = []

# Define leaf size range
leaf_size_range_start = 50
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 50

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the Ball Tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc=f"Querying (Leaf Size {leaf_size:,})"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of Ball Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 50, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 50): 100%|██████████| 1000/1000 [00:07<00:00, 136.70it/s]


-------------------------------------------------------------
Build Time: 6.1852 sec, Search Time: 7.2549 sec
Leaf Size: 50, Total Time: 13.4401 sec

2: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 100): 100%|██████████| 1000/1000 [00:07<00:00, 140.60it/s]


-------------------------------------------------------------
Build Time: 4.9366 sec, Search Time: 7.0502 sec
Leaf Size: 100, Total Time: 11.9868 sec

3: === Testing with Leaf Size: 150, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


...

-------------------------------------------------------------
Build Time: 2.1078 sec, Search Time: 18.6573 sec
Leaf Size: 9,900, Total Time: 20.7651 sec

199: === Testing with Leaf Size: 9,950, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 9,950): 100%|██████████| 1000/1000 [00:18<00:00, 52.74it/s]


-------------------------------------------------------------
Build Time: 2.0893 sec, Search Time: 18.8109 sec
Leaf Size: 9,950, Total Time: 20.9001 sec

200: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 1,000 queries ===


Querying (Leaf Size 10,000): 100%|██████████| 1000/1000 [00:18<00:00, 52.91it/s]

-------------------------------------------------------------
Build Time: 2.1013 sec, Search Time: 18.7493 sec
Leaf Size: 10,000, Total Time: 20.8505 sec

=== Summary of Ball Tree Node Size Test ===
Leaf Size: 50, Build Time: 6.1852 sec, Search Time: 7.2549 sec, Total Time: 13.4401 sec
Leaf Size: 100, Build Time: 4.9366 sec, Search Time: 7.0502 sec, Total Time: 11.9868 sec
Leaf Size: 150, Build Time: 4.3488 sec, Search Time: 6.8055 sec, Total Time: 11.1543 sec
Leaf Size: 200, Build Time: 4.0367 sec, Search Time: 6.7710 sec, Total Time: 10.8077 sec
Leaf Size: 250, Build Time: 3.8262 sec, Search Time: 6.7346 sec, Total Time: 10.5608 sec
Leaf Size: 300, Build Time: 3.7143 sec, Search Time: 6.7147 sec, Total Time: 10.4291 sec
Leaf Size: 350, Build Time: 3.5873 sec, Search Time: 6.7798 sec, Total Time: 10.3672 sec
Leaf Size: 400, Build Time: 3.4819 sec, Search Time: 6.7163 sec, Total Time: 10.1982 sec
Leaf Size: 450, Build Time: 3.5116 sec, Search Time: 6.8132 sec, Total Time: 10.3247 sec
L

**-------Ball Tree Optimum Node Size----------**

In [3]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([   50,  100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
   650,  700,  750,  800,  850,  900,  950, 1000, 1050, 1100, 1150, 1200,
  1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800,
  1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400,
  2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800, 2850, 2900, 2950, 3000,
  3050, 3100, 3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600,
  3650, 3700, 3750, 3800, 3850, 3900, 3950, 4000, 4050, 4100, 4150, 4200,
  4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800,
  4850, 4900, 4950, 5000, 5050, 5100, 5150, 5200, 5250, 5300, 5350, 5400,
  5450, 5500, 5550, 5600, 5650, 5700, 5750, 5800, 5850, 5900, 5950, 6000,
  6050, 6100, 6150, 6200, 6250, 6300, 6350, 6400, 6450, 6500, 6550, 6600,
  6650, 6700, 6750, 6800, 6850, 6900, 6950, 7000, 7050, 7100, 7150, 7200,
  7250, 7300, 7350, 7400, 7450, 7500, 7550, 7600, 7650, 7700, 7750, 7800,
  7850, 7900, 7950, 8000, 8050, 8100, 8150, 8200, 8250, 8300, 8350, 8400,
  8450, 8500, 8550, 8600, 8650, 8700, 8750, 8800, 8850, 8900, 8950, 9000,
  9050, 9100, 9150, 9200, 9250, 9300, 9350, 9400, 9450, 9500, 9550, 9600,
  9650, 9700, 9750, 9800, 9850, 9900, 9950,10000])

# Total times from two runs (or versions)
total_times_1 = np.array([13.9723,12.0212,11.3066,10.9296,10.7704,10.4415,10.2914,10.0993,10.0292,
 10.0091,10.0376, 9.9929, 9.8983, 9.854 , 9.9133, 9.8764, 9.832 , 9.916 ,
  9.8243, 9.9462, 9.9826, 9.9485,10.0456,10.1384,10.1172,10.266 ,10.2657,
 10.2695,10.4671,10.4535,10.5371,10.4426,10.6324,10.6081,10.6745,10.759 ,
 10.7573,10.8751,11.0114,11.0258,11.1606,11.2014,11.1851,11.2075,11.415 ,
 11.5383,11.5033,11.5077,11.5145,11.6327,11.6871,11.827 ,11.9022,12.0425,
 12.0797,12.0773,12.1417,12.1392,12.2953,12.381 ,12.4978,12.5605,12.4443,
 12.6133,12.6154,12.6742,12.8267,12.8682,12.8875,13.0487,12.9943,13.0107,
 13.1879,13.2238,13.2746,13.2631,13.2096,13.2296,13.3562,13.5008,13.6773,
 13.7025,13.6916,13.8879,13.9098,13.9534,13.989 ,14.1374,14.2267,14.2296,
 14.2772,14.3132,14.4803,14.4016,14.5503,14.4553,14.5895,14.6935,14.8016,
 14.8277,15.1056,15.3962,15.3077,15.3129,15.3543,15.444 ,15.5987,16.2219,
 15.5613,15.758 ,15.7652,15.7493,16.0214,16.1997,16.0386,16.1775,16.1992,
 16.1788,16.215 ,16.1098,16.3092,16.4646,16.5785,16.6333,16.8522,16.7477,
 16.5659,16.944 ,16.922 ,17.2376,17.3314,17.384 ,17.1028,17.5893,17.4719,
 17.4557,17.6668,17.6666,17.4907,17.6991,17.771 ,17.7963,18.0541,17.8976,
 17.8851,18.2034,18.3234,18.2335,18.3698,18.1516,18.479 ,18.6465,18.6506,
 18.7037,18.6476,18.6785,18.7676,18.9508,18.7549,19.0346,18.8476,18.9957,
 19.0208,19.0195,18.7813,18.8011,18.842 ,18.9825,18.8463,19.0532,19.1849,
 19.4858,19.646 ,19.5674,19.6352,19.7332,19.8109,19.8762,19.7982,19.8213,
 19.7772,19.8587,19.9283,19.6951,19.7073,19.6892,19.6339,19.8444,19.9347,
 19.9772,20.1376,20.0351,20.071 ,19.9603,20.1156,20.1233,20.1654,20.2971,
 20.319 ,20.27  ])

total_times_2 = np.array([13.4401,11.9868,11.1543,10.8077,10.5608,10.4291,10.3672,10.1982,10.3247,
 10.0877,10.036 , 9.9704, 9.8669, 9.9405, 9.9809,10.0141,10.0066,10.0046,
 10.0886,10.0174,10.0637, 9.9469, 9.9701, 9.9587,10.1114,10.208 ,10.2538,
 10.3694,10.3819,10.5445,10.5681,10.5916,10.6114,10.6533,10.7434,10.6424,
 10.8906,10.8372,10.9487,10.8945,11.0552,11.1535,11.2712,11.2258,11.2979,
 11.4474,11.477 ,11.5128,11.5813,11.7199,11.8168,11.7919,11.8116,11.9951,
 12.5436,12.0708,12.2144,12.1786,12.2534,12.3282,12.315 ,12.3722,12.5007,
 12.6979,12.7738,12.7606,12.8378,13.0404,12.9382,12.973 ,13.0171,13.0799,
 13.3545,13.3368,13.3408,13.3419,13.3642,13.4882,13.5779,13.7174,13.6066,
 13.7833,13.8077,13.9083,13.9374,14.0807,14.245 ,14.3907,14.4193,14.2954,
 14.4689,14.5681,14.7009,14.851 ,14.8278,14.6043,14.6823,14.9001,14.9569,
 14.9071,14.9299,14.9125,14.989 ,15.1017,15.0443,14.8503,15.2839,15.4457,
 15.4933,15.5696,15.6382,15.7277,15.8049,15.7673,15.8774,15.9149,16.0371,
 16.1946,16.3575,16.334 ,16.4452,16.3516,16.3269,16.2719,16.8363,16.8754,
 16.8784,16.9823,17.0205,17.3444,17.2554,17.475 ,17.3991,17.69  ,17.6158,
 17.7178,17.8716,17.9317,17.8967,17.8692,17.9802,18.0707,18.2004,18.2452,
 18.431 ,18.3899,18.3955,18.5993,18.6306,18.7009,18.7329,18.9081,18.9608,
 18.7982,18.9427,19.1405,19.1422,19.2378,19.2283,19.0678,19.3596,19.3044,
 19.3383,19.2277,19.2575,19.3277,19.309 ,19.2177,19.3225,19.3713,19.3691,
 19.7645,19.6695,19.8294,19.7288,19.74  ,20.0498,20.031 ,20.0468,20.0791,
 20.0768,20.0444,20.1199,20.1291,20.0465,20.2604,20.1617,20.3633,20.6266,
 20.6581,20.5417,20.5663,20.666 ,20.7231,20.6792,20.7462,20.7312,20.7651,
 20.9001,20.8505])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 9.8826
Minimum found at leaf size: 650


In [4]:
build_time_1 = np.array([6.6163,4.892 ,4.336 ,4.0303,3.8944,3.6864,3.562 ,3.4863,3.4062,3.367 ,
 3.3067,3.2458,3.1975,3.1324,3.1195,3.0831,3.0468,3.0517,2.9792,3.0058,
 2.9457,2.922 ,2.9108,2.8956,2.8946,2.8732,2.9003,2.8366,2.829 ,2.7998,
 2.8044,2.7921,2.7934,2.7529,2.768 ,2.7408,2.7417,2.7055,2.7064,2.6845,
 2.6624,2.6824,2.6608,2.637 ,2.613 ,2.6625,2.634 ,2.6123,2.6458,2.5985,
 2.5833,2.5849,2.5802,2.5884,2.5689,2.5264,2.5566,2.5248,2.5279,2.5822,
 2.5199,2.5239,2.5042,2.4983,2.4648,2.5064,2.4875,2.4831,2.4654,2.473 ,
 2.4394,2.4599,2.453 ,2.4138,2.4599,2.4327,2.4085,2.4426,2.4101,2.4129,
 2.3845,2.4259,2.3837,2.3979,2.4118,2.3956,2.396 ,2.3729,2.3525,2.3815,
 2.331 ,2.3736,2.3768,2.3403,2.3496,2.3803,2.2932,2.3778,2.3525,2.3207,
 2.3706,2.3853,2.4082,2.3936,2.4051,2.4027,2.4467,2.4335,2.4121,2.3569,
 2.3853,2.3504,2.3716,2.3722,2.3279,2.3295,2.3071,2.3171,2.2849,2.3491,
 2.3145,2.2932,2.2991,2.2863,2.29  ,2.3006,2.2785,2.2725,2.2751,2.2947,
 2.2585,2.2623,2.2426,2.2834,2.2672,2.27  ,2.2352,2.251 ,2.2471,2.2266,
 2.245 ,2.2297,2.265 ,2.222 ,2.1949,2.2291,2.1931,2.1746,2.2288,2.2152,
 2.1977,2.1739,2.1816,2.1788,2.1934,2.1551,2.178 ,2.1583,2.1789,2.2126,
 2.1352,2.1656,2.1628,2.1661,2.1583,2.1729,2.1565,2.1559,2.1911,2.0895,
 2.1537,2.1549,2.1266,2.1274,2.1703,2.167 ,2.0993,2.1455,2.1382,2.1414,
 2.1194,2.1087,2.108 ,2.0844,2.1248,2.1149,2.1035,2.0611,2.111 ,2.0787,
 2.0807,2.0821,2.0966,2.0693,2.08  ,2.0917,2.0664,2.1444,2.0787,2.0802])

build_time_2 = np.array([6.1852,4.9366,4.3488,4.0367,3.8262,3.7143,3.5873,3.4819,3.5116,3.3411,
 3.3113,3.2441,3.1983,3.1733,3.1212,3.0887,3.061 ,2.9924,3.0236,2.9965,
 2.9841,2.9035,2.9426,2.898 ,2.8595,2.8551,2.8203,2.8549,2.8098,2.808 ,
 2.7828,2.7621,2.7747,2.7534,2.7109,2.7495,2.737 ,2.7242,2.6828,2.6768,
 2.6774,2.6605,2.6693,2.6538,2.6415,2.6154,2.6344,2.6375,2.6451,2.6275,
 2.5997,2.5999,2.5348,2.5855,2.5588,2.549 ,2.54  ,2.5268,2.5154,2.499 ,
 2.5143,2.5263,2.5172,2.4919,2.5148,2.4818,2.4416,2.4958,2.4809,2.4531,
 2.4641,2.4398,2.4796,2.4601,2.4425,2.4429,2.4428,2.4317,2.4312,2.4383,
 2.4067,2.4262,2.4132,2.4038,2.4013,2.4121,2.4113,2.436 ,2.3841,2.3506,
 2.3606,2.3782,2.3288,2.3681,2.3666,2.3616,2.3535,2.3837,2.3428,2.3226,
 2.3285,2.2977,2.3427,2.3743,2.3354,2.2872,2.3451,2.3325,2.3265,2.3473,
 2.3047,2.3302,2.3292,2.2917,2.3504,2.2799,2.3185,2.2723,2.2865,2.2867,
 2.2861,2.3011,2.2685,2.2821,2.272 ,2.2871,2.257 ,2.3063,2.2581,2.2721,
 2.2318,2.2705,2.2592,2.2579,2.2477,2.2218,2.2361,2.239 ,2.2621,2.2241,
 2.2296,2.1921,2.2298,2.2281,2.2432,2.2081,2.2071,2.2165,2.2043,2.195 ,
 2.198 ,2.1816,2.1852,2.1915,2.1642,2.208 ,2.1786,2.1766,2.16  ,2.1827,
 2.1988,2.1786,2.1704,2.1832,2.1588,2.18  ,2.1288,2.1637,2.1416,2.1625,
 2.1429,2.1197,2.1375,2.1481,2.1397,2.1175,2.1398,2.133 ,2.1153,2.1306,
 2.1224,2.1223,2.1342,2.1147,2.1042,2.1035,2.1261,2.1277,2.1544,2.1468,
 2.1155,2.116 ,2.1271,2.1138,2.101 ,2.1014,2.0935,2.1078,2.0893,2.1013])

avg_build_times = (build_time_1 + build_time_2) / 2

print(avg_build_times)

[6.40075 4.9143  4.3424  4.0335  3.8603  3.70035 3.57465 3.4841  3.4589
 3.35405 3.309   3.24495 3.1979  3.15285 3.12035 3.0859  3.0539  3.02205
 3.0014  3.00115 2.9649  2.91275 2.9267  2.8968  2.87705 2.86415 2.8603
 2.84575 2.8194  2.8039  2.7936  2.7771  2.78405 2.75315 2.73945 2.74515
 2.73935 2.71485 2.6946  2.68065 2.6699  2.67145 2.66505 2.6454  2.62725
 2.63895 2.6342  2.6249  2.64545 2.613   2.5915  2.5924  2.5575  2.58695
 2.56385 2.5377  2.5483  2.5258  2.52165 2.5406  2.5171  2.5251  2.5107
 2.4951  2.4898  2.4941  2.46455 2.48945 2.47315 2.46305 2.45175 2.44985
 2.4663  2.43695 2.4512  2.4378  2.42565 2.43715 2.42065 2.4256  2.3956
 2.42605 2.39845 2.40085 2.40655 2.40385 2.40365 2.40445 2.3683  2.36605
 2.3458  2.3759  2.3528  2.3542  2.3581  2.37095 2.32335 2.38075 2.34765
 2.32165 2.34955 2.3415  2.37545 2.38395 2.37025 2.34495 2.3959  2.383
 2.3693  2.3521  2.345   2.3403  2.3504  2.33195 2.33915 2.3047  2.3128
 2.2947  2.2857  2.3179  2.3003  2.29715 2.2838  2.2842  2

In [5]:
search_time_1 = np.array([ 7.356 , 7.1292, 6.9705, 6.8993, 6.876 , 6.7551, 6.7294, 6.6131, 6.623 ,
  6.6421, 6.7309, 6.7471, 6.7009, 6.7216, 6.7939, 6.7933, 6.7852, 6.8642,
  6.8451, 6.9404, 7.0369, 7.0265, 7.1348, 7.2428, 7.2226, 7.3928, 7.3654,
  7.4329, 7.6381, 7.6536, 7.7327, 7.6505, 7.839 , 7.8552, 7.9065, 8.0182,
  8.0156, 8.1695, 8.305 , 8.3412, 8.4982, 8.519 , 8.5243, 8.5705, 8.802 ,
  8.8759, 8.8693, 8.8954, 8.8687, 9.0342, 9.1038, 9.2421, 9.322 , 9.4541,
  9.5108, 9.5509, 9.5851, 9.6143, 9.7675, 9.7988, 9.9779,10.0366, 9.9401,
 10.1149,10.1507,10.1679,10.3393,10.3851,10.4222,10.5757,10.5549,10.5508,
 10.7349,10.81  ,10.8147,10.8305,10.8011,10.787 ,10.9461,11.088 ,11.2928,
 11.2766,11.308 ,11.49  ,11.498 ,11.5578,11.593 ,11.7645,11.8742,11.8481,
 11.9462,11.9395,12.1035,12.0613,12.2007,12.075 ,12.2963,12.3157,12.4491,
 12.507 ,12.735 ,13.0109,12.8996,12.9193,12.9492,13.0413,13.152 ,13.7883,
 13.1492,13.4011,13.3799,13.3989,13.6498,13.8276,13.7107,13.848 ,13.8921,
 13.8616,13.9302,13.7608,13.9947,14.1714,14.2793,14.347 ,14.5622,14.4472,
 14.2874,14.6716,14.6469,14.9428,15.073 ,15.1216,14.8602,15.3059,15.2047,
 15.1857,15.4316,15.4156,15.2436,15.4725,15.5259,15.5666,15.7891,15.6757,
 15.6902,15.9743,16.1303,16.0589,16.141 ,15.9364,16.2813,16.4727,16.469 ,
 16.5249,16.4542,16.5233,16.5897,16.7924,16.576 ,16.8221,16.7124,16.8301,
 16.858 ,16.8534,16.6229,16.6282,16.6855,16.8265,16.6552,16.9638,17.0311,
 17.3309,17.5193,17.44  ,17.4648,17.5661,17.7116,17.7307,17.66  ,17.6799,
 17.6578,17.7499,17.8203,17.6106,17.5825,17.5743,17.5304,17.7833,17.8237,
 17.8985,18.057 ,17.953 ,17.9744,17.891 ,18.0356,18.0315,18.099 ,18.1527,
 18.2403,18.1898])

search_time_2 = np.array([ 7.2549, 7.0502, 6.8055, 6.771 , 6.7346, 6.7147, 6.7798, 6.7163, 6.8132,
  6.7465, 6.7247, 6.7263, 6.6686, 6.7673, 6.8597, 6.9254, 6.9456, 7.0122,
  7.065 , 7.0209, 7.0795, 7.0435, 7.0275, 7.0608, 7.2519, 7.3528, 7.4335,
  7.5145, 7.5721, 7.7365, 7.7853, 7.8295, 7.8368, 7.8999, 8.0325, 7.893 ,
  8.1536, 8.1129, 8.266 , 8.2177, 8.3778, 8.493 , 8.6019, 8.572 , 8.6564,
  8.8321, 8.8426, 8.8753, 8.9361, 9.0924, 9.2171, 9.192 , 9.2769, 9.4096,
  9.9848, 9.5217, 9.6744, 9.6518, 9.738 , 9.8292, 9.8008, 9.8459, 9.9835,
 10.206 ,10.259 ,10.2787,10.3962,10.5446,10.4573,10.5199,10.553 ,10.6401,
 10.8749,10.8767,10.8983,10.899 ,10.9214,11.0565,11.1467,11.2791,11.1999,
 11.3571,11.3944,11.5045,11.5361,11.6686,11.8337,11.9547,12.0352,11.9449,
 12.1083,12.1899,12.3721,12.483 ,12.4612,12.2427,12.3288,12.5164,12.6141,
 12.5845,12.6014,12.6148,12.6462,12.7274,12.709 ,12.5631,12.9388,13.1132,
 13.1668,13.2222,13.3335,13.3975,13.4756,13.4757,13.5271,13.635 ,13.7185,
 13.9223,14.071 ,14.0473,14.159 ,14.0504,14.0584,13.9898,14.5644,14.5883,
 14.6214,14.6761,14.7624,15.0723,15.0235,15.2046,15.1399,15.4321,15.3681,
 15.4959,15.6355,15.6927,15.6346,15.645 ,15.7506,15.8786,15.9706,16.0171,
 16.1878,16.1817,16.1883,16.3828,16.4263,16.5059,16.5349,16.7265,16.7756,
 16.6066,16.7785,16.9325,16.9636,17.0612,17.0683,16.8851,17.1609,17.1259,
 17.1679,17.0445,17.0987,17.1478,17.1802,17.054 ,17.1808,17.2088,17.2262,
 17.6448,17.5321,17.6813,17.5891,17.6224,17.91  ,17.8979,17.9315,17.9485,
 17.9544,17.9221,17.9857,18.0144,17.9423,18.1569,18.0357,18.2356,18.4722,
 18.5113,18.4263,18.4503,18.539 ,18.6092,18.5782,18.6447,18.6377,18.6573,
 18.8109,18.7493])

avg_search_times = (search_time_1 + search_time_2) / 2

print(avg_search_times)

[ 7.30545  7.0897   6.888    6.83515  6.8053   6.7349   6.7546   6.6647
  6.7181   6.6943   6.7278   6.7367   6.68475  6.74445  6.8268   6.85935
  6.8654   6.9382   6.95505  6.98065  7.0582   7.035    7.08115  7.1518
  7.23725  7.3728   7.39945  7.4737   7.6051   7.69505  7.759    7.74
  7.8379   7.87755  7.9695   7.9556   8.0846   8.1412   8.2855   8.27945
  8.438    8.506    8.5631   8.57125  8.7292   8.854    8.85595  8.88535
  8.9024   9.0633   9.16045  9.21705  9.29945  9.43185  9.7478   9.5363
  9.62975  9.63305  9.75275  9.814    9.88935  9.94125  9.9618  10.16045
 10.20485 10.2233  10.36775 10.46485 10.43975 10.5478  10.55395 10.59545
 10.8049  10.84335 10.8565  10.86475 10.86125 10.92175 11.0464  11.18355
 11.24635 11.31685 11.3512  11.49725 11.51705 11.6132  11.71335 11.8596
 11.9547  11.8965  12.02725 12.0647  12.2378  12.27215 12.33095 12.15885
 12.31255 12.41605 12.5316  12.54575 12.6682  12.81285 12.7729  12.82335
 12.8291  12.8022  13.0454  13.45075 13.158   13.31165 13.